## Bibliotecas utilizadas

In [1]:
import pandas as pd
import math
from tabulate import tabulate
import numpy as np
from decimal import Decimal, ROUND_HALF_UP

## LENDO BASE CONSOLIDADA

In [2]:
caminho = r"C:/Users/gui-b/Teste-tecnico/teste-tecnico/bases/tratadas/BASE_CONSOLIDADA.csv"
df = pd.read_csv(caminho, encoding="utf-8-sig")

In [3]:
df.head()

,APS,APS.1,Competência concessão,Espécie,Espécie.1,CID,CID.1,Despacho,Despacho.1,Dt Nascimento,...,Vínculo dependentes,Forma Filiação,UF,Qt SM RMI,Ramo Atividade,Dt DCB,Dt DDB,Dt DIB,País de Acordo Internacional,Classificador PA
0,2001420,02001420-Aps Cacimbinhas,202503,31,Auxílio Doenca Previdenciário,S626,S62.6 Frat de Outr Dedos,64.0,Concessao com Analise Documental,1973-11-27,...,Não Informado,Segurado Especial,Alagoas,1.000,Rural,2024-11-25 00:00:00,2025-03-07,2024-09-28,{ñ class},Sem Pensâo Alimentícia
1,2001200,02001200-Aps Girau do Ponciano,202503,31,Auxílio Doenca Previdenciário,M159,M15.9 Poliartrose Ne,64.0,Concessao com Analise Documental,1978-04-06,...,Não Informado,Segurado Especial,Alagoas,1.000,Rural,2025-08-01 00:00:00,2025-03-07,2025-02-03,{ñ class},Sem Pensâo Alimentícia
2,2001060,02001060-Aps Palmeira dos Indios,202503,31,Auxílio Doenca Previdenciário,M545,M54.5 Dor Lombar Baixa,0.0,Concessao Normal,1976-05-12,...,Não Informado,Desempregado,Alagoas,1.414,Comerciario,2025-05-30 00:00:00,2025-03-07,2025-02-07,{ñ class},Sem Pensâo Alimentícia
3,2001010,02001010-Aps Arapiraca,202503,31,Auxílio Doenca Previdenciário,F411,F41.1 Ansiedade Generalizada,0.0,Concessao Normal,1983-07-08,...,Não Informado,Desempregado,Alagoas,2.658,Comerciario,2025-02-28 00:00:00,2025-03-06,2024-11-26,{ñ class},Sem Pensâo Alimentícia
4,2001050,02001050-Aps Maceio - Tabuleiro do Martins,202503,31,Auxílio Doenca Previdenciário,S320,S32.0 Frat de Vertebra Lombar,64.0,Concessao com Analise Documental,1999-03-09,...,Não Informado,Empregado,Alagoas,1.059,Comerciario,2025-01-14 00:00:00,2025-03-06,2024-10-25,{ñ class},Sem Pensâo Alimentícia


## Parte 1: Identificação das Cidades Mais Atrativas

In [4]:
def parse_munres(x):
    if isinstance(x, str):
        parts = x.split("-")
        if len(parts) >= 3:
            uf = parts[1]
            mun = "-".join(parts[2:])
            return f"{mun.strip()} - {uf.strip()}"
    return None

df["municipio_uf"] = df["Mun Resid"].apply(parse_munres)

# filtrando os meses solicitados no case
df = df[df["Competência concessão"].isin([202501, 202502, 202503])]

# contando as concessões por cidade e mês
contagem = df.groupby(["municipio_uf","Competência concessão"]).size().reset_index(name="qtde")

# consolidando total e média
tabela = contagem.groupby("municipio_uf")["qtde"].agg(total_3m="sum", media_mes="mean").reset_index()

# analisando o top 30 cidades
top30 = tabela.sort_values(["total_3m","media_mes"], ascending=False).head(30)
print(top30)

                    municipio_uf  total_3m     media_mes
4989              São Paulo - SP     77136  25712.000000
4198         Rio de Janeiro - RJ     37445  12481.666667
716                Brasília - DF     19996   6665.333333
547          Belo Horizonte - MG     16405   5468.333333
4293               Salvador - BA     14862   4954.000000
1790              Fortaleza - CE     13690   4563.333333
1455               Curitiba - PR     11772   3924.000000
1993              Guarulhos - SP     10187   3395.666667
1897                Goiânia - GO      9963   3321.000000
3888           Porto Alegre - RS      9927   3309.000000
2843                 Manaus - AM      9562   3187.333333
4071                 Recife - PE      8889   2963.000000
5317             Uberlândia - MG      8722   2907.333333
907                Campinas - SP      8190   2730.000000
927            Campo Grande - MS      7796   2598.666667
2556              Joinville - SC      7169   2389.666667
2576            João Pessoa - P

## 1) Quantos clientes são necessários para o breakeven mensal

In [5]:
# ==========================
# PARÂMETROS DO NEGÓCIO
# ==========================
INVESTIMENTO_INICIAL = 100_000.00
CUSTO_FIXO_MENSAL = 15_000.00
RECEITA_POR_CLIENTE = 150.00
CUSTO_POR_CLIENTE = 50.00
MARGEM_CONTRIB = RECEITA_POR_CLIENTE - CUSTO_POR_CLIENTE 
CLIENTES_BREAKEVEN = math.ceil(CUSTO_FIXO_MENSAL / MARGEM_CONTRIB)
print(f"Clientes necessários para breakeven mensal: {CLIENTES_BREAKEVEN}")

Clientes necessários para breakeven mensal: 150


## 2) Considerando uma taxa constante de novas concessões igual à média mensal observada nos meses analisados, em quanto tempo os 30 pontos de agência indicados na Parte 1 teriam o seu breakeven, caso não houvesse perda de benefícios?

In [ ]:
# ==========================
# Tempo de Breakeven
# ==========================

top = top30.copy() 
top["clientes_necessarios_BE_mensal"] = CLIENTES_BREAKEVEN
top["meses_ate_breakeven"] = (CLIENTES_BREAKEVEN / top["media_mes"]).apply(
    lambda x: math.ceil(x) if x > 0 else float("inf")
)

# ordena e ranqueia
top = top.sort_values(["total_3m", "media_mes"], ascending=False).head(30).reset_index(drop=True)
top.insert(0, "Rank", top.index + 1)

apres = numerico.copy()
apres["Total (3m)"] = apres["total_3m"].apply(fmt_int_br)
apres["Média/mês"] = apres["media_mes"].apply(fmt_int_br)
apres["BE (clientes)"] = apres["clientes_necessarios_BE_mensal"].apply(fmt_int_br)
apres["Meses p/ BE"] = apres["meses_ate_breakeven"].astype(int)

apres = apres[["Rank", "municipio_uf", "Total (3m)", "Média/mês", "BE (clientes)", "Meses p/ BE"]]
apres = apres.rename(columns={"municipio_uf": "Município – UF"})

print(tabulate(apres, headers="keys", tablefmt="psql", showindex=False))

+--------+----------------------------+--------------+-------------+-----------------+---------------+
|   Rank | Município – UF             |   Total (3m) |   Média/mês |   BE (clientes) |   Meses p/ BE |
|--------+----------------------------+--------------+-------------+-----------------+---------------|
|      1 | São Paulo - SP             |       77.136 |      25.712 |             150 |             1 |
|      2 | Rio de Janeiro - RJ        |       37.445 |      12.482 |             150 |             1 |
|      3 | Brasília - DF              |       19.996 |       6.665 |             150 |             1 |
|      4 | Belo Horizonte - MG        |       16.405 |       5.468 |             150 |             1 |
|      5 | Salvador - BA              |       14.862 |       4.954 |             150 |             1 |
|      6 | Fortaleza - CE             |       13.69  |       4.563 |             150 |             1 |
|      7 | Curitiba - PR              |       11.772 |       3.924 |     

# Parte 3: Consideração da Duração dos Benefícios
Considerando que um benefício temporário (Espécie = auxílios) tem uma duração média de 3 meses, enquanto que um beneficiário permanente (todos com exceção dos auxílios) vivem em média até os 80 anos, recalcule o tempo necessário para que os 30 pontos de agência indicados na Parte 1 atinjam o breakeven, assumindo que a base de clientes de cada agência evolui de acordo com a média de inclusões dos meses analisados e considerando a rotatividade de beneficiários temporários.


In [ ]:
MESES_ALVO = {202501, 202502, 202503}


if "fmt_int_br" not in globals():
    def fmt_int_br(x):  
        return f"{int(round(x)):,}".replace(",", ".")

def _to_datetime(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s, errors="coerce")

if "parse_munres" not in globals():
    def parse_munres(x):
        if isinstance(x, str):
            parts = x.split("-")
            if len(parts) >= 3 and parts[0] != "00000":
                uf = parts[1].strip()
                mun = "-".join(parts[2:]).strip()
                return f"{mun} - {uf}"
        return None

def _is_auxilio_series(s: pd.Series) -> pd.Series:
    return s.astype(str).str.contains(r"aux[ií]lio", case=False, regex=True)

df3 = df.copy()

if "competencia" not in df3.columns:
    comp = df3["Competência concessão"].astype(str).str.extract(r"(\d{6})")[0]
    df3["competencia"] = pd.to_numeric(comp, errors="coerce").astype("Int64")

df3 = df3[df3["competencia"].isin(MESES_ALVO)].copy()

# municipio_uf 
if "municipio_uf" not in df3.columns:
    df3["municipio_uf"] = df3["Mun Resid"].apply(parse_munres)
df3 = df3[df3["municipio_uf"].notna()].copy()

# flag para estou usando para os auxílio
if "is_auxilio" not in df3.columns:
    df3["is_auxilio"] = _is_auxilio_series(df3["Espécie"])

if "idade_anos" not in df3.columns:
    df3["dt_dib"] = _to_datetime(df3["Dt DIB"])
    df3["dt_nasc"] = _to_datetime(df3["Dt Nascimento"])
    df3["idade_anos"] = ((df3["dt_dib"] - df3["dt_nasc"]).dt.days / 365.25).astype("float")

# incluindo as funções dos churns
def _prepara_fluxos_cidade(df_city: pd.DataFrame):
    # médias para jan-mar/25
    m_total = df_city.groupby("competencia").size().mean()
    m_aux = df_city[df_city["is_auxilio"]].groupby("competencia").size().mean() if df_city["is_auxilio"].any() else 0.0
    m_perm = (m_total or 0.0) - (m_aux or 0.0)

    # permanentes: vida útil até 80 anos 
    perm = df_city[~df_city["is_auxilio"]].copy()
    if "idade_anos" in perm.columns and len(perm) > 0 and not perm["idade_anos"].isna().all():
        perm["months_left_80"] = ((80 - perm["idade_anos"]).clip(lower=0)) * 12
        def survival_share(t: int) -> float:
            return (perm["months_left_80"] >= t).mean()
    else:
        def survival_share(t: int) -> float:
            return 1.0

    return float(m_perm or 0.0), float(m_aux or 0.0), survival_share

def _base_ativa_churn(m_perm: float, m_aux: float, survival_share, t: int) -> float:
    perm_active = sum(m_perm * survival_share(k) for k in range(1, t + 1))
    aux_active  = m_aux * min(t, 3)  # auxílios duram 3 meses
    return perm_active + aux_active

def _meses_breakeven_churn(df_city: pd.DataFrame) -> int:
    m_perm, m_aux, survival_share = _prepara_fluxos_cidade(df_city)
    for t in range(1, 121):  # até 10 anos
        ativos = _base_ativa_churn(m_perm, m_aux, survival_share, t)
        if MARGEM_CONTRIB * ativos >= CUSTO_FIXO_MENSAL:
            return t
    return math.inf

resultados = []
for city in top30["municipio_uf"]:
    sub = df3[df3["municipio_uf"] == city].copy()
    t_be = _meses_breakeven_churn(sub)
    resultados.append((city, t_be))

df_be_churn = pd.DataFrame(resultados, columns=["municipio_uf", "meses_break_even_churn"])

top30_churn = top30.merge(df_be_churn, on="municipio_uf", how="left")
top30_churn = top30_churn.sort_values(["meses_break_even_churn", "total_3m"], ascending=[True, False]).reset_index(drop=True)
top30_churn.insert(0, "Rank (churn)", top30_churn.index + 1)

apres_churn = top30_churn[["Rank (churn)", "municipio_uf", "total_3m", "media_mes", "meses_break_even_churn"]].copy()
apres_churn = apres_churn.rename(columns={
    "municipio_uf": "Município – UF",
    "total_3m": "Total (3m)",
    "media_mes": "Média/mês",
    "meses_break_even_churn": "Meses p/ BE (churn)"
})
apres_churn["Total (3m)"] = apres_churn["Total (3m)"].apply(fmt_int_br)
apres_churn["Média/mês"] = apres_churn["Média/mês"].apply(fmt_int_br)

print(tabulate(apres_churn, headers="keys", tablefmt="psql", showindex=False))

+----------------+----------------------------+--------------+-------------+-----------------------+
|   Rank (churn) | Município – UF             |   Total (3m) |   Média/mês |   Meses p/ BE (churn) |
|----------------+----------------------------+--------------+-------------+-----------------------|
|              1 | São Paulo - SP             |       77.136 |      25.712 |                     1 |
|              2 | Rio de Janeiro - RJ        |       37.445 |      12.482 |                     1 |
|              3 | Brasília - DF              |       19.996 |       6.665 |                     1 |
|              4 | Belo Horizonte - MG        |       16.405 |       5.468 |                     1 |
|              5 | Salvador - BA              |       14.862 |       4.954 |                     1 |
|              6 | Fortaleza - CE             |       13.69  |       4.563 |                     1 |
|              7 | Curitiba - PR              |       11.772 |       3.924 |               

# Desafio Opcional: Novo Produto de Crédito
O banco pretende implementar um novo produto de crédito voltado apenas para beneficiários permanentes que possuem perfil de crédito. As condições do produto são as seguintes:

●	Beneficiários permanentes do sexo masculino até 60 anos de idade: Liberação de até R$ 20.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

●	Beneficiários permanentes do sexo masculino até 70 anos de idade: Liberação de até R$ 15.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

●	Beneficiários permanentes do sexo masculino até 80 anos de idade: Liberação de até R$ 10.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

●	Beneficiários permanentes do sexo feminino até 60 anos de idade: Liberação de até R$ 20.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

●	Beneficiários permanentes do sexo feminino até 70 anos de idade: Liberação de até R$ 10.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

●	Beneficiários permanentes do sexo feminino até 80 anos de idade: Liberação de até R$ 10.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

●	Beneficiários permanentes do sexo feminino até 85 anos de idade: Liberação de até R$ 5.000,00, com uma taxa de 1,6% a.m e um prazo de 30 meses.

Considerando que 20% dos clientes que possuem perfil de crédito contratarão o empréstimo em sua totalidade (nas 30 agências das localidades sugeridas): 


# D1. 
Qual será a receita que o banco terá com este empréstimo, para o grupo que será incluído no primeiro mês de funcionamento da agência, no prazo total do produto? 

In [ ]:

total_principal = int(
    (elegiveis["qtd_20pct_int"] * elegiveis["faixa"].map(PRINCIPAL_FAIXA)).sum()
)
total_receita = int(elegiveis["receita_20_int"].sum())

resumo_total = pd.DataFrame({
    "Métrica": [
        "Tomadores efetivos (20%)",
        "Principal concedido (R$)",
        "Receita de juros em 30m (R$)"
    ],
    "Valor": [
        f"{total_tomadores:,}".replace(",", "."),
        f"{total_principal:,}".replace(",", "."),
        f"{total_receita:,}".replace(",", "."),
    ]
})

print(tabulate(resumo_total, headers="keys", tablefmt="psql", showindex=False))


resumo_cidade = (
    elegiveis.groupby("municipio_uf")
    .apply(lambda g: pd.Series({
        "Tomadores (20%)": int(g["qtd_20pct_int"].sum()),
        "Principal (R$)": int((g["qtd_20pct_int"] * g["faixa"].map(PRINCIPAL_FAIXA)).sum()),
        "Receita 30m (R$)": int(g["receita_20_int"].sum()),
    }))
    .reset_index()
)

resumo_cidade_fmt = resumo_cidade.copy()
for col in ["Principal (R$)", "Receita 30m (R$)"]:
    resumo_cidade_fmt[col] = resumo_cidade_fmt[col].map(lambda x: f"{x:,}".replace(",", "."))

print("\n== Receita e Principal por Município (20% de tomadores) ==")
print(tabulate(resumo_cidade_fmt.head(10), headers="keys", tablefmt="psql", showindex=False))


+------------------------------+-------------+
| Métrica                      | Valor       |
|------------------------------+-------------|
| Tomadores efetivos (20%)     | 23.646      |
| Principal concedido (R$)     | 420.620.000 |
| Receita de juros em 30m (R$) | 112.286.725 |
+------------------------------+-------------+

== Receita e Principal por Município (20% de tomadores) ==
+----------------------+-------------------+------------------+--------------------+
| municipio_uf         |   Tomadores (20%) | Principal (R$)   | Receita 30m (R$)   |
|----------------------+-------------------+------------------+--------------------|
| Belo Horizonte - MG  |              1089 | 19.405.000       | 5.180.268          |
| Belém - PA           |               415 | 7.360.000        | 1.964.789          |
| Brasília - DF        |              1333 | 23.980.000       | 6.401.587          |
| Campinas - SP        |               543 | 9.420.000        | 2.514.719          |
| Campo Grande -

C:\Users\gui-b\AppData\Local\Temp\ipykernel_31936\804296604.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


# D2. 
Considerando que 5% dos clientes que contratam o produto não pagam nenhuma parcela, qual será o lucro do banco em 30 meses, para o grupo de clientes considerado acima?

In [ ]:
##Lucro do banco com 5% de inadimplência total

DEFAULT_RATE = 0.05  # 5% inadimplência

# perdas: 5% dos clientes * (principal + juros) por contrato
elegiveis["perda_default"] = elegiveis.apply(
    lambda r: round_half_up_int(
        r["qtd_20pct_int"] * DEFAULT_RATE * (PRINCIPAL_FAIXA[r["faixa"]] + r["juros_unit"])
    ),
    axis=1
)

# lucro = receita bruta - perdas
elegiveis["lucro_liquido"] = elegiveis["receita_20_int"] - elegiveis["perda_default"]


lucro_total = int(elegiveis["lucro_liquido"].sum())

resumo_D2 = pd.DataFrame({
    "Métrica": ["Lucro líquido em 30m (com 5% default)"],
    "Valor": [f"{lucro_total:,}".replace(",", ".")]
})


print(tabulate(resumo_D2, headers="keys", tablefmt="psql", showindex=False))

resumo_cidade_D2 = (
    elegiveis.groupby("municipio_uf")
    .apply(lambda g: pd.Series({
        "Tomadores (20%)": int(g["qtd_20pct_int"].sum()),
        "Principal (R$)": int((g["qtd_20pct_int"] * g["faixa"].map(PRINCIPAL_FAIXA)).sum()),
        "Receita 30m (R$)": int(g["receita_20_int"].sum()),
        "Perda c/ 5% default (R$)": int(g["perda_default"].sum()),
        "Lucro líquido (R$)": int(g["lucro_liquido"].sum()),
    }))
    .reset_index()
)

if "round_half_up_int" not in globals():
    
    def round_half_up_int(x):
        return int(Decimal(str(x)).quantize(Decimal('1'), rounding=ROUND_HALF_UP))


resumo_cidade_D2_fmt = resumo_cidade_D2.copy()
for col in ["Principal (R$)", "Receita 30m (R$)", "Perda c/ 5% default (R$)", "Lucro líquido (R$)"]:
    resumo_cidade_D2_fmt[col] = resumo_cidade_D2_fmt[col].map(lambda x: f"{x:,}".replace(",", "."))

print("\n== Lucro líquido por município (20% tomadores, 5% inadimplência) ==")
print(tabulate(resumo_cidade_D2_fmt, headers="keys", tablefmt="psql", showindex=False))  # <-- sem head(10)


with pd.ExcelWriter("D2_resultados.xlsx") as writer:
    
    resumo_D2.to_excel(writer, index=False, sheet_name="D2_total")
    
    resumo_cidade_D2.to_excel(writer, index=False, sheet_name="D2_por_municipio")

print("Arquivo gerado: D2_resultados.xlsx (abas: D2_total, D2_por_municipio)")


+---------------------------------------+------------+
| Métrica                               | Valor      |
|---------------------------------------+------------|
| Lucro líquido em 30m (com 5% default) | 85.641.392 |
+---------------------------------------+------------+

== Lucro líquido por município (20% tomadores, 5% inadimplência) ==
+----------------------------+-------------------+------------------+--------------------+----------------------------+----------------------+
| municipio_uf               |   Tomadores (20%) | Principal (R$)   | Receita 30m (R$)   | Perda c/ 5% default (R$)   | Lucro líquido (R$)   |
|----------------------------+-------------------+------------------+--------------------+----------------------------+----------------------|
| Belo Horizonte - MG        |              1089 | 19.405.000       | 5.180.268          | 1.229.264                  | 3.951.004            |
| Belém - PA                 |               415 | 7.360.000        | 1.964.789     

C:\Users\gui-b\AppData\Local\Temp\ipykernel_31936\707888577.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [ ]:

# ---------------- Parâmetros ----------------
TAKE_UP = 0.20
MESES_ALVO = {202501, 202502, 202503}
TAXA_AO_MES = 0.016
PRAZO_MESES = 30

def round_half_up_int(x):
    return int(Decimal(str(x)).quantize(Decimal('1'), rounding=ROUND_HALF_UP))

def pmt(rate: float, nper: int, pv: float) -> float:
    if rate == 0:
        return pv / nper
    return rate * pv / (1 - (1 + rate) ** (-nper))

def faixa_rotulo(sexo: str, idade: float) -> str:
    s = (sexo or "").strip().lower()
    if pd.isna(idade): return "inelegível"
    if s.startswith("m"):
        if idade <= 60: return "Masculino - até 60 anos"
        if idade <= 70: return "Masculino - até 70 anos"
        if idade <= 80: return "Masculino - até 80 anos"
        return "inelegível"
    if s.startswith("f"):
        if idade <= 60: return "Feminino - até 60 anos"
        if idade <= 70: return "Feminino - até 70 anos"
        if idade <= 80: return "Feminino - até 80 anos"
        if idade <= 85: return "Feminino - até 85 anos"
        return "inelegível"
    return "inelegível"

def parse_munres(x):
    if isinstance(x, str):
        parts = x.split("-")
        if len(parts) >= 3 and parts[0] != "00000":
            uf = parts[1].strip()
            mun = "-".join(parts[2:]).strip()
            return f"{mun} - {uf}"
    return None

# principal por faixa
PRINCIPAL_FAIXA = {
    "Masculino - até 60 anos": 20000.0,
    "Masculino - até 70 anos": 15000.0,
    "Masculino - até 80 anos": 10000.0,
    "Feminino - até 60 anos": 20000.0,
    "Feminino - até 70 anos": 10000.0,
    "Feminino - até 80 anos": 10000.0,
    "Feminino - até 85 anos":  5000.0,
}

# juros para as faixas
JUROS_UNIT_FAIXA = {
    k: (pmt(TAXA_AO_MES, PRAZO_MESES, v) * PRAZO_MESES - v)
    for k, v in PRINCIPAL_FAIXA.items()
}


base = df.copy()

# competencia
if "competencia" not in base.columns:
    comp = base["Competência concessão"].astype(str).str.extract(r"(\d{6})")[0]
    base["competencia"] = pd.to_numeric(comp, errors="coerce").astype("Int64")

# municipio_uf
if "municipio_uf" not in base.columns:
    base["municipio_uf"] = base["Mun Resid"].apply(parse_munres)

# sexo
if "Sexo." in base.columns:
    base["sexo_norm"] = base["Sexo."].astype(str).str.lower()
elif "Sexo" in base.columns:
    base["sexo_norm"] = base["Sexo"].astype(str).str.lower()
else:
    base["sexo_norm"] = ""

# idade
if "idade_anos" not in base.columns:
    base["dt_dib"] = pd.to_datetime(base["Dt DIB"], errors="coerce")
    base["dt_nasc"] = pd.to_datetime(base["Dt Nascimento"], errors="coerce")
    base["idade_anos"] = ((base["dt_dib"] - base["dt_nasc"]).dt.days / 365.25).astype("float")

# permanentes
if "is_auxilio" not in base.columns:
    base["Espécie"] = base["Espécie"].astype(str)
    base["is_auxilio"] = base["Espécie"].str.contains(r"aux[ií]lio", case=False, regex=True)

# filtrando os top30 + meses + permanentes para incluir ao código
top_keys = set(top30["municipio_uf"].tolist())
base = base[
    base["municipio_uf"].isin(top_keys)
    & base["municipio_uf"].notna()
    & base["competencia"].isin(MESES_ALVO)
    & (~base["is_auxilio"])
].copy()

# classificando as faixas e descartaando os  inelegíveis
base["faixa"] = base.apply(lambda r: faixa_rotulo(r.get("sexo_norm",""), r.get("idade_anos", np.nan)), axis=1)
base = base[base["faixa"] != "inelegível"].copy()

grp = (
    base.groupby(["municipio_uf", "faixa", "competencia"])
        .size()
        .reset_index(name="qtd_mes")
)

elegiveis = (
    grp.groupby(["municipio_uf", "faixa"])["qtd_mes"]
       .mean()
       .reset_index(name="qtd_elegiveis")
)

elegiveis["qtd_20pct_int"] = elegiveis["qtd_elegiveis"].apply(lambda x: round_half_up_int(x * TAKE_UP))


elegiveis["juros_unit"] = elegiveis["faixa"].map(JUROS_UNIT_FAIXA)
elegiveis["receita_20_int"] = elegiveis.apply(
    lambda r: round_half_up_int(r["qtd_20pct_int"] * r["juros_unit"]),
    axis=1
)

# Quantidades
pivot_qtd_20 = elegiveis.pivot_table(
    index="municipio_uf", columns="faixa", values="qtd_20pct_int", aggfunc="sum", fill_value=0
).reset_index()

# Receita
pivot_rec_20 = elegiveis.pivot_table(
    index="municipio_uf", columns="faixa", values="receita_20_int", aggfunc="sum", fill_value=0
).reset_index()

# garantir colunas e ordem
colunas = [
    "Município",
    "Masculino - até 60 anos",
    "Masculino - até 70 anos",
    "Masculino - até 80 anos",
    "Feminino - até 60 anos",
    "Feminino - até 70 anos",
    "Feminino - até 80 anos",
    "Feminino - até 85 anos",
]

def reordena(df_in):
    df_out = df_in.rename(columns={"municipio_uf": "Município"}).copy()
    for c in colunas[1:]:
        if c not in df_out.columns:
            df_out[c] = 0
    return df_out[colunas]

pivot_qtd_20 = reordena(pivot_qtd_20)
pivot_rec_20 = reordena(pivot_rec_20)

# ---------------- Exporta um único XLSX com 2 abas ----------------
with pd.ExcelWriter("qtd_rct_por_municipio_faixas.xlsx") as writer:
    pivot_qtd_20.to_excel(writer, index=False, sheet_name="Qtd_20")
    pivot_rec_20.to_excel(writer, index=False, sheet_name="Receita_20")

print("Arquivo gerado: qtd_rct_por_municipio_faixas.xlsx")


Arquivo gerado: quantidade_e_receita_20pct_por_municipio_faixa.xlsx


In [ ]:
def preview_tab(df: pd.DataFrame, titulo: str, max_rows: int = 20, add_total: bool = True, fmt_thousands: bool = True):
    if df is None or not isinstance(df, pd.DataFrame):
        print(f"[{titulo}] DataFrame não encontrado.")
        return
    view = df.copy()

    if add_total:
        tot = {"Município": "TOTAL"}
        for c in view.columns[1:]:
            if pd.api.types.is_numeric_dtype(view[c]):
                tot[c] = view[c].sum()
            else:
                tot[c] = ""
        view = pd.concat([view, pd.DataFrame([tot])], ignore_index=True)

    if fmt_thousands:
        for c in view.columns[1:]:
            if pd.api.types.is_numeric_dtype(view[c]):
                view[c] = view[c].map(lambda x: f"{int(x):,}".replace(",", ".") if pd.notna(x) else x)

    print(f"\n== {titulo} ==")
    print(tabulate(view.head(max_rows), headers="keys", tablefmt="psql", showindex=False))

preview_tab(pivot_qtd_20, "Quantidades (20%) por Município e Faixa", max_rows=35)
preview_tab(pivot_rec_20, "Receita (20%) por Município e Faixa (R$)", max_rows=35)


== Quantidades (20%) por Município e Faixa ==
+----------------------------+---------------------------+---------------------------+---------------------------+--------------------------+--------------------------+--------------------------+--------------------------+
| Município                  |   Masculino - até 60 anos |   Masculino - até 70 anos |   Masculino - até 80 anos |   Feminino - até 60 anos |   Feminino - até 70 anos |   Feminino - até 80 anos |   Feminino - até 85 anos |
|----------------------------+---------------------------+---------------------------+---------------------------+--------------------------+--------------------------+--------------------------+--------------------------|
| Belo Horizonte - MG        |                   320     |                   121     |                         5 |                  473     |                  148     |                       18 |                        4 |
| Belém - PA                 |                   145     |   